In [1]:
import os
from datetime import datetime

# Define the target directory (change yours)
TARGET_DIRECTORY = r"C:\Users\pablosal\Desktop\gbbai-azure-ai-capacity"

# Check if the directory exists
if os.path.exists(TARGET_DIRECTORY):
    # Change the current working directory
    os.chdir(TARGET_DIRECTORY)
    print(f"Directory changed to {os.getcwd()}")
else:
    print(f"Directory {TARGET_DIRECTORY} does not exist.")

Directory changed to C:\Users\pablosal\Desktop\gbbai-azure-ai-capacity


In [2]:
# Import the AOAICapacityChecker class from the capacity module
from src.capacity.aoai import AOAICapacityChecker

## Authentication Methods

There are three ways to authenticate with Azure to fetch AOAI capacity information from your subscription. Below are the supported methods:

### 1. Using DefaultAzureCredential (Recommended)

This method is recommended as it automatically handles various authentication mechanisms such as environment variables, Managed Identity, and Azure CLI credentials.

```python
from aoai_capacity_checker import AOAICapacityChecker

checker = AOAICapacityChecker(subscription_id="your-subscription-id")
```

### 2. Using Client ID, Client Secret, and Tenant ID

You can authenticate by passing a **Client ID**, **Client Secret**, and **Tenant ID** explicitly. This method is suitable when working with a service principal.

```python
from aoai_capacity_checker import AOAICapacityChecker

checker = AOAICapacityChecker(
    subscription_id="your-subscription-id",
    client_id="your-client-id",
    client_secret="your-client-secret",
    tenant_id="your-tenant-id"
)
```

### 3. Using a Provided Bearer Token

If you already have a bearer token, you can pass it directly into the checker.

```python
from aoai_capacity_checker import AOAICapacityChecker

checker = AOAICapacityChecker(
    subscription_id="your-subscription-id",
    bearer_token="your-bearer-token"
)
```

We are using app registration for the most control and production setup. Please refer to the [HOW-TO.md](HOW-TO.md) file for detailed instructions on how to obtain the Client ID, Client Secret, and Tenant ID for your specific application.

In [3]:
# Set your subscription ID directly
subscription_id = "1a4bb722-f155-4502-8033-022a9eb1481b"

# Retrieve client credentials from environment variables for security
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")
tenant_id = os.getenv("TENANT_ID")

# Ensure that the environment variables are set
if not client_id or not client_secret or not tenant_id:
    raise EnvironmentError(
        "Please set the CLIENT_ID, CLIENT_SECRET, and TENANT_ID environment variables."
    )

# Initialize AOAICapacityChecker with the retrieved credentials
aoai_capacity_checker = AOAICapacityChecker(
    subscription_id=subscription_id,
    client_id=client_id,
    client_secret=client_secret,
    tenant_id=tenant_id,
)

2024-09-05 11:14:31,631 - micro - MainProcess - INFO     Using ClientSecretCredential to retrieve the bearer token. (aoai.py:_get_bearer_token_via_credentials:79)


In [4]:
# Fetch capacity data for the GPT-4o model
capacity_data_gpt4o = aoai_capacity_checker.get_model_capacity(
    model_name="gpt-4o", model_version="2024-05-13"
)

# Fetch capacity data for the GPT-4o-mini model
capacity_data_gpt4o_mini = aoai_capacity_checker.get_model_capacity(
    model_name="gpt-4o-mini", model_version="2024-07-18"
)

2024-09-05 11:14:34,180 - micro - MainProcess - INFO     Successfully fetched data. (aoai.py:_make_api_call:98)
2024-09-05 11:14:34,185 - micro - MainProcess - INFO     Cached data for gpt-4o___2024-05-13 (aoai.py:_cache_capacity_data:168)
2024-09-05 11:14:34,188 - micro - MainProcess - INFO     Fetched and cached new data for gpt-4o___2024-05-13 (aoai.py:get_model_capacity:127)
2024-09-05 11:14:35,719 - micro - MainProcess - INFO     Successfully fetched data. (aoai.py:_make_api_call:98)
2024-09-05 11:14:35,721 - micro - MainProcess - INFO     Cached data for gpt-4o-mini___2024-07-18 (aoai.py:_cache_capacity_data:168)
2024-09-05 11:14:35,722 - micro - MainProcess - INFO     Fetched and cached new data for gpt-4o-mini___2024-07-18 (aoai.py:get_model_capacity:127)


In [13]:
# Retrieve all cached capacity data, including both available and fine-tuned capacities.
# Optionally, filter by model name and version.

# Example 1: Retrieve capacity data for a specific model and version
capacity_data_specific_model = aoai_capacity_checker.retrieve_capacity(
    model_name="gpt-4o", model_version="2024-05-13"
)

unique_models_specific = capacity_data_specific_model["Model"].unique()
print("Unique models for specific model and version:", unique_models_specific)

# Example 2: Retrieve capacity data for all models and versions
capacity_data_all_models = aoai_capacity_checker.retrieve_capacity()

unique_models_all = capacity_data_all_models["Model"].unique()
print("Unique models for all models and versions:", unique_models_all)

2024-09-05 10:35:05,812 - micro - MainProcess - INFO     DataFrame created with columns: ['Model', 'SKU', 'Region', 'Available Capacity', 'Fine-tuned Capacity', 'Timestamp', 'Unit'] (aoai.py:retrieve_capacity:349)
2024-09-05 10:35:05,820 - micro - MainProcess - INFO     DataFrame created with columns: ['Model', 'SKU', 'Region', 'Available Capacity', 'Fine-tuned Capacity', 'Timestamp', 'Unit'] (aoai.py:retrieve_capacity:349)


Unique models for specific model and version: ['gpt-4o (v2024-05-13)']
Unique models for all models and versions: ['gpt-4o (v2024-05-13)' 'gpt-4o-mini (v2024-07-18)']



The `check_capacity` function verifies if the available or fine-tuned capacity for a specified model and version meets the required capacity. It allows filtering by SKUs and regions and returns a structured dictionary with detailed information.

#### Benefits:
- **Flexibility**: Allows filtering by model name, version, SKUs, and regions.
- **Detailed Output**: Provides a comprehensive dictionary with availability status, SKU-region combinations, timestamp, and detailed capacity information.
- **Customizable**: Supports checking both available and fine-tuned capacities.

#### Returns:
A dictionary with:
- **availability**: Boolean indicating if the required capacity is met.
- **sku_regions**: List of SKU and region combinations that meet the required capacity.
- **timestamp**: Timestamp of the last update.
- **details**: Detailed information about the capacity in each region.

### Example Usage:



In [5]:
data = aoai_capacity_checker.check_capacity(
    model_name="gpt-4o-mini",
    model_version="2024-07-18",
    required_capacity=25,
    skus="ProvisionedManaged",
    regions=["brazilsouth", "eastus2", "swedencentral"],
)

2024-09-05 11:15:16,989 - micro - MainProcess - INFO     Overriding cached data for gpt-4o-mini___2024-07-18 (aoai.py:get_model_capacity:116)
2024-09-05 11:15:18,493 - micro - MainProcess - INFO     Successfully fetched data. (aoai.py:_make_api_call:98)
2024-09-05 11:15:18,496 - micro - MainProcess - INFO     Cached data for gpt-4o-mini___2024-07-18 (aoai.py:_cache_capacity_data:168)
2024-09-05 11:15:18,498 - micro - MainProcess - INFO     Fetched and cached new data for gpt-4o-mini___2024-07-18 (aoai.py:get_model_capacity:127)


In [6]:
import json

print(json.dumps(data, indent=4))

{
    "availability": true,
    "sku_regions": {
        "ProvisionedManaged": [
            "swedencentral"
        ]
    },
    "timestamp": "2024-09-05 16:15:18 UTC",
    "details": [
        {
            "sku": "ProvisionedManaged",
            "region": "brazilsouth",
            "available_capacity": 11,
            "fine_tuned_capacity": 0,
            "meets_required_capacity": false
        },
        {
            "sku": "ProvisionedManaged",
            "region": "eastus2",
            "available_capacity": 0,
            "fine_tuned_capacity": 0,
            "meets_required_capacity": false
        },
        {
            "sku": "ProvisionedManaged",
            "region": "swedencentral",
            "available_capacity": 100,
            "fine_tuned_capacity": 0,
            "meets_required_capacity": true
        }
    ]
}


## 📊 For Visual Enthusiasts - Visualize the Capacity

In [7]:
# Plot capacity
# Example 1: Plot capacity for all models and versions
aoai_capacity_checker.plot_capacity()

# Example 2: Plot capacity for a specific model
aoai_capacity_checker.plot_capacity(model_name="gpt-4o")

# Example 3: Plot capacity for a specific model and version
aoai_capacity_checker.plot_capacity(model_name="gpt-4o", model_version="2024-05-13")

2024-09-05 11:15:23,671 - micro - MainProcess - INFO     DataFrame created with columns: ['Model', 'SKU', 'Region', 'Available Capacity', 'Fine-tuned Capacity', 'Timestamp', 'Unit'] (aoai.py:retrieve_capacity:349)


2024-09-05 11:15:25,762 - micro - MainProcess - INFO     DataFrame created with columns: ['Model', 'SKU', 'Region', 'Available Capacity', 'Fine-tuned Capacity', 'Timestamp', 'Unit'] (aoai.py:retrieve_capacity:349)


2024-09-05 11:15:25,937 - micro - MainProcess - INFO     DataFrame created with columns: ['Model', 'SKU', 'Region', 'Available Capacity', 'Fine-tuned Capacity', 'Timestamp', 'Unit'] (aoai.py:retrieve_capacity:349)
